In [1]:
import pandas as pd
import io
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
import pickle
print ('Ran')

Ran


In [2]:
app_train = pd.read_csv('D:/Project_Data/application_train.csv')
app_test = pd.read_csv('D:/Project_Data/application_test.csv')

print("Shape of training data: {}".format(app_train.shape))
print("Shape of test data: {}".format(app_test.shape))

Shape of training data: (307511, 122)
Shape of test data: (48744, 121)


In [3]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train_labels = app_train['TARGET']
train_sk_id_curr = app_train['SK_ID_CURR']
test_sk_id_curr = app_test['SK_ID_CURR']

app_train.drop('SK_ID_CURR', inplace=True, axis=1)
app_test.drop('SK_ID_CURR', inplace=True, axis=1)

app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 120)
Testing Features shape:  (48744, 120)


In [5]:
cat_features = [f for f in app_train.columns if app_train[f].dtype == 'object']

In [6]:
def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, query_cols, sorter=sidx)]

In [7]:
cat_features_inds = column_index(app_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_inds)

Cat features are: ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']
[ 0  1  2  3  9 10 11 12 13 26 30 38 84 85 87 88]


In [10]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in cat_features:
    app_train[col] = le.fit_transform(app_train[col].astype(str))
    app_test[col] = le.fit_transform(app_test[col].astype(str))

In [137]:
app_train
app_test

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0,0,0,1,0,135000.0,568800.0,20560.5,450000.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0,1,0,99000.0,222768.0,17370.0,180000.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,0,1,1,1,0,202500.0,663264.0,69777.0,630000.0,7,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,0,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,0,1,1,0,1,180000.0,625500.0,32067.0,625500.0,6,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5,0,0,1,1,0,270000.0,959688.0,34600.5,810000.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0
6,0,1,1,1,2,180000.0,499221.0,22117.5,373500.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
7,0,1,0,1,0,166500.0,180000.0,14220.0,180000.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
8,0,0,0,1,0,315000.0,364896.0,28957.5,315000.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0
9,0,0,1,1,1,162000.0,45000.0,5337.0,45000.0,1,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [138]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')
imputer.fit(app_train)
app_train = imputer.transform(app_train)
app_test = imputer.transform(app_test)

In [139]:
app_train
app_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 3.],
       [0., 1., 1., ..., 0., 1., 4.],
       ...,
       [0., 0., 1., ..., 0., 3., 1.],
       [0., 1., 0., ..., 0., 0., 2.],
       [0., 0., 1., ..., 0., 1., 4.]])

In [140]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score

In [141]:
X_train, X_test, y_train, y_test = train_test_split(app_train, train_labels, test_size=0.2, stratify=train_labels, random_state=1)
print("Postive examples in train set: {}".format(np.sum(y_train==0)))
print("Negative examples in train set: {}".format(np.sum(y_train==1)))

print("Postive examples in test set: {}".format(np.sum(y_test==0)))
print("Negative examples in test set: {}".format(np.sum(y_test==1)))

Postive examples in train set: 226148
Negative examples in train set: 19860
Postive examples in test set: 56538
Negative examples in test set: 4965


clf = CatBoostClassifier(iterations=1000,
                              learning_rate=0.1,
                              depth=7,
                              l2_leaf_reg=40,
                              bootstrap_type='Bernoulli',
                              subsample=0.7,
                              scale_pos_weight=5,
                              eval_metric='AUC',
                              metric_period=50,
                              od_type='Iter',
                              od_wait=45,
                              random_seed=17,
                              allow_writing_files=False)

In [143]:
clf = CatBoostClassifier(iterations=500,
                              learning_rate=0.1,
                              depth=8,
                              loss_function='Logloss',
                              bootstrap_type='Bernoulli',
                              eval_metric='AUC',
                              class_weights=[1, 2],
                              random_seed=9, 
                              verbose=False)

In [144]:
clf.fit(X_train, y_train,
             eval_set=(X_test, y_test),
             cat_features=cat_features_inds,
             use_best_model=True,
             verbose=True)

0:	test: 0.6839530	best: 0.6839530 (0)	total: 608ms	remaining: 5m 3s
1:	test: 0.6986228	best: 0.6986228 (1)	total: 1.07s	remaining: 4m 25s
2:	test: 0.7029762	best: 0.7029762 (2)	total: 1.62s	remaining: 4m 28s
3:	test: 0.7120240	best: 0.7120240 (3)	total: 2.21s	remaining: 4m 34s
4:	test: 0.7161793	best: 0.7161793 (4)	total: 2.77s	remaining: 4m 33s
5:	test: 0.7186024	best: 0.7186024 (5)	total: 3.78s	remaining: 5m 11s
6:	test: 0.7217129	best: 0.7217129 (6)	total: 4.36s	remaining: 5m 6s
7:	test: 0.7226502	best: 0.7226502 (7)	total: 4.92s	remaining: 5m 2s
8:	test: 0.7254107	best: 0.7254107 (8)	total: 5.49s	remaining: 4m 59s
9:	test: 0.7261989	best: 0.7261989 (9)	total: 5.72s	remaining: 4m 40s
10:	test: 0.7295454	best: 0.7295454 (10)	total: 6.28s	remaining: 4m 39s
11:	test: 0.7308561	best: 0.7308561 (11)	total: 6.79s	remaining: 4m 36s
12:	test: 0.7331343	best: 0.7331343 (12)	total: 7.3s	remaining: 4m 33s
13:	test: 0.7340968	best: 0.7340968 (13)	total: 7.85s	remaining: 4m 32s
14:	test: 0.7358

115:	test: 0.7585959	best: 0.7586071 (112)	total: 1m 5s	remaining: 3m 35s
116:	test: 0.7586002	best: 0.7586071 (112)	total: 1m 5s	remaining: 3m 34s
117:	test: 0.7586165	best: 0.7586165 (117)	total: 1m 6s	remaining: 3m 34s
118:	test: 0.7585978	best: 0.7586165 (117)	total: 1m 6s	remaining: 3m 33s
119:	test: 0.7586744	best: 0.7586744 (119)	total: 1m 7s	remaining: 3m 33s
120:	test: 0.7586621	best: 0.7586744 (119)	total: 1m 7s	remaining: 3m 32s
121:	test: 0.7587176	best: 0.7587176 (121)	total: 1m 8s	remaining: 3m 32s
122:	test: 0.7586715	best: 0.7587176 (121)	total: 1m 9s	remaining: 3m 31s
123:	test: 0.7587372	best: 0.7587372 (123)	total: 1m 9s	remaining: 3m 31s
124:	test: 0.7588408	best: 0.7588408 (124)	total: 1m 10s	remaining: 3m 30s
125:	test: 0.7588404	best: 0.7588408 (124)	total: 1m 10s	remaining: 3m 30s
126:	test: 0.7588618	best: 0.7588618 (126)	total: 1m 11s	remaining: 3m 29s
127:	test: 0.7589843	best: 0.7589843 (127)	total: 1m 11s	remaining: 3m 28s
128:	test: 0.7589821	best: 0.75898

225:	test: 0.7616325	best: 0.7616970 (223)	total: 2m 8s	remaining: 2m 35s
226:	test: 0.7616145	best: 0.7616970 (223)	total: 2m 8s	remaining: 2m 35s
227:	test: 0.7617192	best: 0.7617192 (227)	total: 2m 9s	remaining: 2m 34s
228:	test: 0.7616669	best: 0.7617192 (227)	total: 2m 10s	remaining: 2m 34s
229:	test: 0.7616443	best: 0.7617192 (227)	total: 2m 10s	remaining: 2m 33s
230:	test: 0.7616113	best: 0.7617192 (227)	total: 2m 11s	remaining: 2m 32s
231:	test: 0.7615611	best: 0.7617192 (227)	total: 2m 11s	remaining: 2m 32s
232:	test: 0.7614871	best: 0.7617192 (227)	total: 2m 12s	remaining: 2m 31s
233:	test: 0.7614693	best: 0.7617192 (227)	total: 2m 13s	remaining: 2m 31s
234:	test: 0.7614952	best: 0.7617192 (227)	total: 2m 13s	remaining: 2m 30s
235:	test: 0.7614679	best: 0.7617192 (227)	total: 2m 14s	remaining: 2m 30s
236:	test: 0.7614946	best: 0.7617192 (227)	total: 2m 14s	remaining: 2m 29s
237:	test: 0.7615268	best: 0.7617192 (227)	total: 2m 15s	remaining: 2m 29s
238:	test: 0.7615282	best: 0

335:	test: 0.7614345	best: 0.7619074 (257)	total: 3m 12s	remaining: 1m 34s
336:	test: 0.7615113	best: 0.7619074 (257)	total: 3m 13s	remaining: 1m 33s
337:	test: 0.7615107	best: 0.7619074 (257)	total: 3m 13s	remaining: 1m 32s
338:	test: 0.7615359	best: 0.7619074 (257)	total: 3m 14s	remaining: 1m 32s
339:	test: 0.7614182	best: 0.7619074 (257)	total: 3m 14s	remaining: 1m 31s
340:	test: 0.7614423	best: 0.7619074 (257)	total: 3m 15s	remaining: 1m 31s
341:	test: 0.7614381	best: 0.7619074 (257)	total: 3m 16s	remaining: 1m 30s
342:	test: 0.7613887	best: 0.7619074 (257)	total: 3m 16s	remaining: 1m 29s
343:	test: 0.7613787	best: 0.7619074 (257)	total: 3m 17s	remaining: 1m 29s
344:	test: 0.7613623	best: 0.7619074 (257)	total: 3m 17s	remaining: 1m 28s
345:	test: 0.7612647	best: 0.7619074 (257)	total: 3m 18s	remaining: 1m 28s
346:	test: 0.7613022	best: 0.7619074 (257)	total: 3m 18s	remaining: 1m 27s
347:	test: 0.7613774	best: 0.7619074 (257)	total: 3m 19s	remaining: 1m 27s
348:	test: 0.7613696	best

446:	test: 0.7600774	best: 0.7619074 (257)	total: 4m 17s	remaining: 30.5s
447:	test: 0.7600530	best: 0.7619074 (257)	total: 4m 18s	remaining: 30s
448:	test: 0.7601074	best: 0.7619074 (257)	total: 4m 18s	remaining: 29.4s
449:	test: 0.7600696	best: 0.7619074 (257)	total: 4m 19s	remaining: 28.8s
450:	test: 0.7600854	best: 0.7619074 (257)	total: 4m 20s	remaining: 28.3s
451:	test: 0.7600853	best: 0.7619074 (257)	total: 4m 20s	remaining: 27.7s
452:	test: 0.7600726	best: 0.7619074 (257)	total: 4m 21s	remaining: 27.1s
453:	test: 0.7600711	best: 0.7619074 (257)	total: 4m 21s	remaining: 26.5s
454:	test: 0.7601362	best: 0.7619074 (257)	total: 4m 22s	remaining: 26s
455:	test: 0.7601520	best: 0.7619074 (257)	total: 4m 23s	remaining: 25.4s
456:	test: 0.7601238	best: 0.7619074 (257)	total: 4m 23s	remaining: 24.8s
457:	test: 0.7601296	best: 0.7619074 (257)	total: 4m 24s	remaining: 24.2s
458:	test: 0.7601167	best: 0.7619074 (257)	total: 4m 24s	remaining: 23.7s
459:	test: 0.7601022	best: 0.7619074 (257)

In [145]:
from sklearn.metrics import accuracy_score
print('AUC:', roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]))
y_pred = clf.predict(X_test)
print('Accuracy score ',accuracy_score(y_test, y_pred))

AUC: 0.7619074118069474
Accuracy score  0.9160691348389509


In [146]:
predictions = clf.predict(app_test)

In [147]:
submission = pd.DataFrame({'SK_ID_CURR': test_sk_id_curr, 'TARGET': predictions})

In [148]:
submission.head()

,SK_ID_CURR,TARGET
0,100001,0.0
1,100005,0.0
2,100013,0.0
3,100028,0.0
4,100038,0.0


In [149]:
submission.to_csv('baseline_catboost_imputed_median_2.csv', index = False)